# sentiment analysis using cohere

setup paths and common variables

In [2]:
!pip install cohere pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.2/249.2 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.4/388.4 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119

In [8]:
import cohere
import csv
import pandas as pd
import time
import random

check whether API key exists (should be in your environment variable)

In [14]:
# Initialize the Cohere client with your API key
from google.colab import userdata
api_key = userdata.get('COHERE_API_KEY')
co = cohere.ClientV2(api_key=api_key)

In [21]:
# Define the sentiment analysis prompt template
prompt_template = '''
Analyze the sentiment of the Hacker News comment with the context of the AI-related story title found below under "Input text:" whether the comment is negative (-1), neutral (0), or positive (1) towards AI. First understand the meaning of the story title, then understand the meaning of the comment in the context of the story title, then analyze the sentiment of the comments in the context of the story title whether the comment is negative (-1), neutral (0), or positive (1) towards AI, and conclude the sentiment of the comment to only one best sentiment type.

Sentiment can contain only a single number either:
  -1 (for negative towards AI)
  0 (for neutral towards AI)
  1 (for positive towards AI)

# Input text:
Story title: """
{title}
"""
Comment: """
{comment}
"""

# Output format:
Sentiment: your_sentiment
Explanation: your_explanation_for_the_sentiment
'''

# Function to get sentiment using Cohere's API
def get_sentiment_and_explanation(title, comment):
    prompt = prompt_template.format(title=title, comment=comment)

    # Repeats for 3 times in case of errors
    for _ in range(3):
        try:
            # Call the Cohere API with the prompt
            response = co.chat(
                model='command-r-plus-08-2024',
                messages=[
                    {
                        "role": "user",
                        "content": prompt,
                    }
                ],
            )
            # Extract the text from the response
            result = response.message.content[0].text.strip()

            # Extract sentiment and explanation from the response text
            sentiment_line = next((line for line in result.split('\n') if "Sentiment:" in line), None)
            explanation_line = next((line for line in result.split('\n') if "Explanation:" in line), None)

            if sentiment_line and explanation_line:
                sentiment = int(sentiment_line.split(":")[1].strip())
                explanation = explanation_line.split(":")[1].strip()
                return sentiment, explanation
            else:
                raise ValueError("Invalid response format")

            # Extract and return the sentiment
            # return int(response.message.content[0].text) # Convert the sentiment label to an integer
        except Exception as e:
            print(f"API call failed: {e}. Pausing for 1-2 minutes...")
            time.sleep(random.randint(60, 120))

# Function to process comments and save periodically
def process_comments(df, output_csv_file, save_interval=20):
    for i, row in df.iterrows():
        # Skip rows that already have a sentiment and explanation value
        if pd.notna(row.get('sentiment')) and pd.notna(row.get('explanation')):
            continue

        # Perform sentiment analysis and get the explanation
        sentiment, explanation = get_sentiment_and_explanation(row['title'], row['comment_text'])
        df.at[i, 'sentiment'] = sentiment
        df.at[i, 'explanation'] = explanation

        # Save progress to CSV after every `save_interval` rows
        if (i + 1) % save_interval == 0:
            df.to_csv(output_csv_file, index=False)
            print(f"Saved progress at row {i + 1}")

    # Final save at the end of processing
    df.to_csv(output_csv_file, index=False)
    print(f"Final result saved to {output_csv_file}")



In [ ]:
# Load the CSV file with the Hacker News data
csv_file = 'sentiment_385_sampled_cleanup.csv'  # Replace with the actual path to your CSV file
df = pd.read_csv(csv_file)

# If the 'sentiment' and 'explanation' columns don't exist, add them
if 'sentiment' not in df.columns:
    df['sentiment'] = None
if 'explanation' not in df.columns:
    df['explanation'] = None

# Output file to save the results
output_csv_file = 'hackernews_comments_with_sentiment_COHERE.csv'

# Process the comments and periodically save the results
process_comments(df, output_csv_file)

print(f'Sentiment analysis completed and saved to {output_csv_file}')

API call failed: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 10 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions". Pausing for 1-2 minutes...
